## Q16.1

这里定义一个二叉树来进行计算。通过递归可以计算出0时刻期权的价格为56。

In [1]:
class Node:
    """A binary tree node.

    Args:
        stock: stock price at this node
        u: up rate
        d: down rate
        er: risk-free rate of interest
    """
    def __init__(self, stock, u, d, er):
        self.stock  = stock
        self.u      = u
        self.d      = d
        self.er     = er

        self.parent = None      ## parent node
        self.left   = None      ## left child node (the up branch)
        self.right  = None      ## right child node (the down branch)
    
    def is_leaf(self):
        return self.left == None and self.right == None
    
    @property
    def q(self):
        return (self.er - self.d) / (self.u - self.d)
    
    @property
    def up_branch(self):
        if self.left == None:
            self.left = Node(self.stock * self.u, self.u, self.d, self.er)
            self.left.parent = self
        
        return self.left
    
    @property
    def down_branch(self):
        if self.right == None:
            self.right = Node(self.stock * self.d, self.u, self.d, self.er)
            self.right.parent = self
        
        return self.right

In [2]:
## information at t = 0
stock = 100
u = 2.0
d = 0.5
er = 1.25

## initialize the binary tree structure
## t = 0
root = Node(stock, u, d, er)

## t = 1
t1 = [root.up_branch, root.down_branch]

## t = 2
t2 = []
for node in t1:
    t2.append(node.up_branch)
    t2.append(node.down_branch)

## option function
def Option(node):
    if node.is_leaf():
        s2 = node.stock
        s1 = node.parent.stock
        s0 = node.parent.parent.stock

        return max(s2 - min([s0, s1, s2]), 0)
    
    q  = node.q
    er = node.er

    return (q * Option(node.up_branch) + (1-q) * Option(node.down_branch)) / er


print(f"The option price at t=0 is {Option(root):.2f}.")

The option price at t=0 is 56.00.


## Q16.2

这里通过动态规划来求解期权的价格。

In [3]:
class Node:
    """A binary tree node.

    Args:
        stock: stock price at this node
        u: up rate
        d: down rate
        er: risk-free rate of interest
    """
    def __init__(self, stock, u, d, er):
        self.stock  = stock
        self.u      = u
        self.d      = d
        self.er     = er

        self._q     = None      ## risk-neutral probability
        self._price = None      ## asset price

        self.left   = None      ## left child node (the up branch)
        self.right  = None      ## right child node (the down branch)
    
    def is_leaf(self):
        return self.left == None and self.right == None
    
    @property
    def q(self):
        if self._q is None:
            self._q = (self.er - self.d) / (self.u - self.d)
        
        return self._q
    
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, value):
        self._price = value
    
    @property
    def up_branch(self):
        return self.left
    
    @up_branch.setter
    def up_branch(self, node):
        self.left = node
    
    @property
    def down_branch(self):
        return self.right
    
    @down_branch.setter
    def down_branch(self, node):
        self.right = node

In [4]:
def create_layer(parents):
    """Create a layer of nodes with given parents.
    """

    nodes = []

    up_node = None

    for pa in parents:
        ## retrieve information from pa
        stock = pa.stock
        u     = pa.u
        d     = pa.d
        er    = pa.er

        ## initialize up branch
        if up_node is None:
            up_node = Node(stock * u, u, d, er)
            nodes.append(up_node)
        
        ## create down branch from pa
        down_node = Node(stock * d, u, d, er)

        ## link pa with up_node and down_node
        pa.up_branch   = up_node
        pa.down_branch = down_node

        ## switch down_node to up_node
        up_node = down_node

        ## add to nodes list
        nodes.append(up_node)

    return nodes

def create_pyramid(n_layer, stock, u, d, er):
    """Create a pyramid (binary tree) of nodes, and return its root.
    """

    ## initialize the root
    root = Node(stock, u, d, er)

    nodes = [root]

    for i in range(n_layer-1):
        nodes = create_layer(nodes)

    return root

In [5]:
## option function generator
def OptionGenerator(option_func):
    """An option generator that creates the option price function.
    """

    def Option(node):
        """Option price function
        """

        ## return option price if we have found it
        if node.price is not None:
            return node.price

        ## otherwise solve it recursively
        if node.is_leaf():
            node.price = option_func(node)
        else:
            q  = node.q
            er = node.er

            node.price = (q * Option(node.up_branch) + (1-q) * Option(node.down_branch)) / er

        return node.price
    
    return Option

In [6]:
import math
from functools import partial

stock = 2.341           ## stock price today

sigma   = 17.7 / 100    ## price volatility
delta_t = 246           ## number of trading days
r       = 2.75 / 100    ## risk-free interest rate
duration= 49            ## periods

## constants
u = math.exp(sigma * math.sqrt(1 / delta_t))
d = 1.0 / u
er= math.exp(r / delta_t)
K = 2.5

## option price function
def EuropeanCall(S, K):
    return max(S - K, 0.)

Option = OptionGenerator(lambda node: partial(EuropeanCall, K=K)(node.stock))

## build the pyramid
root = create_pyramid(duration+1, stock, u, d, er)

print(f"The option price today is: {Option(root):.6f}.")

The option price today is: 0.024637.
